
1. [Install Treno](https://github.com/erosmontin/treno)
1. FS pipeline definition


In [14]:
!! pip install git+https://github.com/erosmontin/treno #--force-reinstall
!! pip install xgboost
!! pip install imbalanced-learn


['Requirement already satisfied: imbalanced-learn in /data/softwares/installations/miniconda/envs/std/lib/python3.12/site-packages (0.13.0)',
 'Requirement already satisfied: numpy<3,>=1.24.3 in /data/softwares/installations/miniconda/envs/std/lib/python3.12/site-packages (from imbalanced-learn) (2.3.1)',
 'Requirement already satisfied: scipy<2,>=1.10.1 in /data/softwares/installations/miniconda/envs/std/lib/python3.12/site-packages (from imbalanced-learn) (1.16.0)',
 'Requirement already satisfied: scikit-learn<2,>=1.3.2 in /data/softwares/installations/miniconda/envs/std/lib/python3.12/site-packages (from imbalanced-learn) (1.6.0)',
 'Requirement already satisfied: sklearn-compat<1,>=0.1 in /data/softwares/installations/miniconda/envs/std/lib/python3.12/site-packages (from imbalanced-learn) (0.1.3)',
 'Requirement already satisfied: joblib<2,>=1.1.1 in /data/softwares/installations/miniconda/envs/std/lib/python3.12/site-packages (from imbalanced-learn) (1.4.2)',
 'Requirement alread

In [15]:
import numpy as np
import pandas as pd

def generate_fake_data(n_samples=100, n_features=20, n_groups=5, classification=True, random_state=None):
    """
    Generates fake data for testing feature selection and classification.

    Args:
        n_samples (int): Number of samples to generate.
        n_features (int): Number of features to generate.
        n_groups (int): Number of groups to generate (if grouping is enabled).
        classification (bool): If True, generates classification labels (0 or 1).
                               If False, generates regression labels.
        random_state (int): Seed for random number generation.

    Returns:
        tuple: A tuple containing:
            - x (pd.DataFrame): Generated features.
            - y (pd.Series): Generated labels.
            - groups (pd.Series or None): Generated groups, or None if n_groups is 0 or None.
    """
    if random_state is not None:
        np.random.seed(random_state)

    # Generate features
    x = pd.DataFrame(np.random.rand(n_samples, n_features), columns=[f'feature_{i}' for i in range(n_features)])

    # Generate labels
    if classification:
        # Generate binary labels
        y = pd.Series(np.random.randint(0, 2, n_samples), name='label')
    else:
        # Generate regression labels based on a simple linear combination of features
        weights = np.random.rand(n_features)
        y = pd.Series(np.dot(x, weights) + np.random.randn(n_samples) * 0.5, name='label')

    # Generate groups
    groups = None
    if n_groups and n_groups > 0:
        groups = pd.Series(np.random.randint(0, n_groups, n_samples), name='group')

    return x, y, groups

# Example usage:
# Generate classification data with groups
X_cls, y_cls, groups_cls = generate_fake_data(n_samples=200, n_features=30, n_groups=10, classification=True, random_state=42)
print("Classification data shape (X, y, groups):", X_cls.shape, y_cls.shape, groups_cls.shape)
display(X_cls.head())
display(y_cls.head())
display(groups_cls.head())

# Generate regression data without groups
X_reg, y_reg, groups_reg = generate_fake_data(n_samples=150, n_features=25, classification=False, random_state=42)
print("\nRegression data shape (X, y, groups):", X_reg.shape, y_reg.shape, groups_reg)
display(X_reg.head())
display(y_reg.head())

Classification data shape (X, y, groups): (200, 30) (200,) (200,)


,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29
0,0.374540,0.950714,0.731994,0.598658,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,...,0.611853,0.139494,0.292145,0.366362,0.456070,0.785176,0.199674,0.514234,0.592415,0.046450
1,0.607545,0.170524,0.065052,0.948886,0.965632,0.808397,0.304614,0.097672,0.684233,0.440152,...,0.969585,0.775133,0.939499,0.894827,0.597900,0.921874,0.088493,0.195983,0.045227,0.325330
2,0.388677,0.271349,0.828738,0.356753,0.280935,0.542696,0.140924,0.802197,0.074551,0.986887,...,0.863103,0.623298,0.330898,0.063558,0.310982,0.325183,0.729606,0.637557,0.887213,0.472215
3,0.119594,0.713245,0.760785,0.561277,0.770967,0.493796,0.522733,0.427541,0.025419,0.107891,...,0.289751,0.161221,0.929698,0.808120,0.633404,0.871461,0.803672,0.186570,0.892559,0.539342
4,0.807440,0.896091,0.318003,0.110052,0.227935,0.427108,0.818015,0.860731,0.006952,0.510747,...,0.962447,0.251782,0.497249,0.300878,0.284840,0.036887,0.609564,0.502679,0.051479,0.278646


0    0
1    0
2    0
3    0
4    0
Name: label, dtype: int64

0    9
1    7
2    9
3    0
4    7
Name: group, dtype: int64


Regression data shape (X, y, groups): (150, 25) (150,) 0      1
1      3
2      3
3      4
4      3
      ..
145    1
146    0
147    4
148    0
149    1
Name: group, Length: 150, dtype: int64


,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24
0,0.374540,0.950714,0.731994,0.598658,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,...,0.183405,0.304242,0.524756,0.431945,0.291229,0.611853,0.139494,0.292145,0.366362,0.456070
1,0.785176,0.199674,0.514234,0.592415,0.046450,0.607545,0.170524,0.065052,0.948886,0.965632,...,0.122038,0.495177,0.034389,0.909320,0.258780,0.662522,0.311711,0.520068,0.546710,0.184854
2,0.969585,0.775133,0.939499,0.894827,0.597900,0.921874,0.088493,0.195983,0.045227,0.325330,...,0.542696,0.140924,0.802197,0.074551,0.986887,0.772245,0.198716,0.005522,0.815461,0.706857
3,0.729007,0.771270,0.074045,0.358466,0.115869,0.863103,0.623298,0.330898,0.063558,0.310982,...,0.119594,0.713245,0.760785,0.561277,0.770967,0.493796,0.522733,0.427541,0.025419,0.107891
4,0.031429,0.636410,0.314356,0.508571,0.907566,0.249292,0.410383,0.755551,0.228798,0.076980,...,0.871461,0.803672,0.186570,0.892559,0.539342,0.807440,0.896091,0.318003,0.110052,0.227935


0    7.031183
1    5.910845
2    6.405714
3    6.118606
4    5.894284
Name: label, dtype: float64

In [21]:
from treno.utils import (
    remove_nans,
    zScoreFeatures,
    filterFeaturesByMAD,
    filterFeaturesByScore,
    filterFeaturesByCorrelation,
    rankFeaturesByRepeatedGini
)
from sklearn.ensemble import RandomForestClassifier

def classification_feature_selection(
    features,
    targets,
    groups=None,
    score_threshold=0.5,
    corr_threshold=0.9,
    score_model=None,
    score_test_size=0.3,
    score_n_repeats=1,
    gini_n_repeats=10,
    gini_test_size=0.1,
    gini_random_seed=None,
    return_gini=False
):
    """
    Feature selection pipeline for classification.

    Parameters:
        features: DataFrame or ndarray of features
        targets: Series or ndarray of targets
        groups: Optional, group labels for splitting
        score_threshold: Threshold for feature score selection
        corr_threshold: Correlation threshold for feature removal
        score_model: scikit-learn classifier to use for scoring (default: RandomForestClassifier)
        score_test_size: Test size for score evaluation
        score_n_repeats: Number of repeats for score evaluation
        gini_n_repeats: Number of repeats for Gini ranking
        gini_test_size: Test size for Gini ranking
        gini_random_seed: Random seed for Gini ranking
        return_gini: If True, also return Gini scores

    Returns:
        DataFrame of selected features (optionally sorted by Gini), and optionally Gini scores
    """
    # 1. Remove NaNs
    features, targets = remove_nans(features, targets)

    # 2. Normalize features
    features = zScoreFeatures(features)

    # 3. Filter by MAD
    features = filterFeaturesByMAD(features)
    if features.shape[1] == 0:
        raise ValueError("No features passed MAD filter")

    # 4. Filter by score (user-selected model or RandomForestClassifier)
    model = score_model if score_model is not None else RandomForestClassifier()
    features, score_values = filterFeaturesByScore(
        features,
        targets,
        groups=groups,
        threshold=score_threshold,
        return_score=True,
        model=model,
        test_size=score_test_size,
        n_repeats=score_n_repeats
    )
    if features.shape[1] == 0:
        raise ValueError("No features passed score filter")

    # 5. Filter by correlation
    features = filterFeaturesByCorrelation(
        features,
        threshold=corr_threshold,
        score=score_values.values
    )
    if features.shape[1] == 0:
        raise ValueError("No features passed correlation filter")

    # 6. Rank by repeated Gini index (optional)
    if return_gini:
        features_sorted, gini_ranks = rankFeaturesByRepeatedGini(
            features, targets,
            n_repeats=gini_n_repeats,
            test_size=gini_test_size,
            random_seed=gini_random_seed,
            groups=groups,
            return_gini=True
        )
        return features_sorted, gini_ranks
    else:
        features_sorted = rankFeaturesByRepeatedGini(
            features, targets,
            n_repeats=gini_n_repeats,
            test_size=gini_test_size,
            random_seed=gini_random_seed,
            groups=groups,
            return_gini=False
        )
        return features_sorted

In [ ]:
from treno.utils import generate_fake_data
# Generate classification data with groups


# Use the classification data with groups to test the pipeline
x, y, groups = generate_fake_data(n_samples=200, n_features=30, n_groups=10, classification=True, random_state=42)


# Example: Only selected features
selected_features = classification_feature_selection(x, y)
print("Selected features after pipeline:", selected_features.columns.tolist())

# Example: Selected features and Gini scores
selected_features, gini_scores = classification_feature_selection(x, y, return_gini=True)
print("Selected features after pipeline:", selected_features.columns.tolist())
print("Gini scores:\n", gini_scores)

TypeError: 'int' object is not iterable